<a href="https://colab.research.google.com/github/Patisson4/ML/blob/master/test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рубежная контрольная работа

Эту работу выполнять не обязательно. Баллы набранные за работу просуммируются с баллами, которые были набраны за контрольную работу 1 и выставлены как оценка за промежуточную аттестацию "автоматом". 

Сделайте копию ноутбука

Присвойте переменной `v` значение свой табельный номер ИСУ, это нужно для определения вашего варианта. Запустите код в следующей ячейке.

In [ ]:
from datetime import datetime
current_datetime = datetime.now()
print(current_datetime)
v = 311550 # по умолчанию вписан мой номер

2022-12-12 08:46:35.240739


# Общее задание

Нужно решить задачу бинарной классификации, предварительно построив признаковое описание объектов на основе нескольких таблиц. 

Целевая переменная - пол клиента.

В качестве модели нужно использовать нейронную сеть, которую нужно строить с помощью `keras` или `torch` на выбор студента.

# Данные

Для всех последующих заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **transactions, tr_mcc_codes и gender_train** из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк.

Эти данные уже использовались в работе 2. Описание данных тут: https://drive.google.com/file/d/13sn8orjkoGfO1SVZYByXNmZQYFRfmOdp/view?usp=sharing

Скачать данные отсюда: https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L?usp=sharing

## Задание 1

В задании требуется на основе нескольких таблиц с данными сделать признаковое описание объектов. 

Объектами являются клиенты. Клиенты идентифицируются с помощью `customer_id`, которые есть в таблицах **transactions** и **gender_train**. В качестве призаков нужно использовать даннее по категориям транзаций: ```mcc_code``` — mcc-код транзакции есть в таблицах **transactions** и **tr_mcc_codes**


Для выбора варианта, запустите код в следующей ячейке



In [ ]:
print ('Список категорий')
feature_list = ['Одежда (также нужно искать "одежды")', 'Ценные бумаги', 'Магазины косметики и Салоны красоты', 'Образовательные услуги (а также школы, университеты)',
                'Прокат автомобилей', 'Автозапчасти, Автошины и Станции техобслуживания (а также СТО)', 'Видеоигры (искать надо по "видеоигр")',
                'Отели', 'Программирование', 'Ремонт', 'Предсказатели будущего', 'Ювелирные изделия']
if (int(str(v)[4]) % 2) == 0:
  print (feature_list[::2])
else:
  print (feature_list[::-2])


if (int(str(v)[5]) % 2) == 0:
  print ('В качестве значений признаков используйте количество транзакций с тратами и количество транзакций с поступлениями')
else: 
  print ('В качестве значений признаков используйте сумму по транзакциям с тратами и сумму по транзакциям с поступлениями')

Список категорий
['Ювелирные изделия', 'Ремонт', 'Отели', 'Автозапчасти, Автошины и Станции техобслуживания (а также СТО)', 'Образовательные услуги (а также школы, университеты)', 'Ценные бумаги']
В качестве значений признаков используйте количество транзакций с тратами и количество транзакций с поступлениями


Каждая категория может быть связана с одним или несколькими mcc-кодами. Например, вот несколько mcc-кодов по категории "Одежда": 
* 5621; Готовая женская одежда
* 5641; Детская одежда
* 5651; Одежда для всей семьи

Таким образом получится 12 признаков, то есть шесть категорий, каждая из которых даст два значения: траты и поступления. Например, категория "Одежда" даёт признаки "Одежда траты" и "Одежда поступления".

Сделайте нормализацию значений признаков так, чтобы все означения менялись от 0 до 1. 

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

tr_mcc_codes = pd.read_csv("/content/drive/MyDrive/tr_mcc_codes.csv", sep=";")
transactions = pd.read_csv("/content/drive/MyDrive/transactions.csv", sep=",", nrows=1000000)
gender_train = pd.read_csv("/content/drive/MyDrive/gender_train.csv", sep=",")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Вопросы к заданию 1 (6 баллов)

1. Для каждой категории выведите соответствующие ей mcc-коды с описаниями.
2. Существуют ли пользователи, у которых значения всех признаков не определены?
3. Выведите id пользователя, у которого самое большое количество непустых признаков.

## Ответы:

In [ ]:
import numpy as np

# 1
my_mcc_codes = {}
my_mcc_codes['Ювелирные изделия'] = tr_mcc_codes.loc[tr_mcc_codes.mcc_description.str.contains('ювелир', case=False)].mcc_code.values
my_mcc_codes['Ремонт'] = tr_mcc_codes.loc[tr_mcc_codes.mcc_description.str.contains('ремонт', case=False)].mcc_code.values
my_mcc_codes['Отели'] = tr_mcc_codes.loc[tr_mcc_codes.mcc_description.str.contains('отел', case=False)].mcc_code.values

my_mcc_codes['Автозапчасти, Автошины и Станции техобслуживания (а также СТО)'] = tr_mcc_codes.loc[tr_mcc_codes.mcc_description.str.contains('запчаст', case=False)].mcc_code.values
my_mcc_codes['Автозапчасти, Автошины и Станции техобслуживания (а также СТО)'] = \
np.append(my_mcc_codes['Автозапчасти, Автошины и Станции техобслуживания (а также СТО)'], \
tr_mcc_codes.loc[tr_mcc_codes.mcc_description.str.contains('шин', case=False)].mcc_code.values)
my_mcc_codes['Автозапчасти, Автошины и Станции техобслуживания (а также СТО)'] = \
np.append(my_mcc_codes['Автозапчасти, Автошины и Станции техобслуживания (а также СТО)'], \
tr_mcc_codes.loc[tr_mcc_codes.mcc_description.str.contains('Станц|СТО')].mcc_code.values)

my_mcc_codes['Образовательные услуги (а также школы, университеты)'] = tr_mcc_codes.loc[tr_mcc_codes.mcc_description.str.contains('школ[а,ы]', case=False)].mcc_code.values
my_mcc_codes['Образовательные услуги (а также школы, университеты)'] = \
np.append(my_mcc_codes['Образовательные услуги (а также школы, университеты)'], \
tr_mcc_codes.loc[tr_mcc_codes.mcc_description.str.contains('Образовательные услуги', case=False)].mcc_code.values)

my_mcc_codes['Ценные бумаги'] = tr_mcc_codes.loc[tr_mcc_codes.mcc_description.str.contains('ценные бумаги', case=False)].mcc_code.values

for i in my_mcc_codes.values():
  for j in i:
    print(tr_mcc_codes[tr_mcc_codes.mcc_code == j])

# 2


    mcc_code                                    mcc_description
34      5094  Драгоценные камни и металлы, часы и ювелирные ...
     mcc_code                                    mcc_description
101      5944  Магазины по продаже часов, ювелирных изделий и...
    mcc_code                                    mcc_description
62      5511  Легковой и грузовой транспорт — продажа, серви...
     mcc_code                                mcc_description
151      7629  Ремонт бытовой техники, ремонт электроприборы
     mcc_code mcc_description
152      7699    Общий ремонт
   mcc_code                 mcc_description
7      3501  Жилье — отели, мотели, курорты
     mcc_code                                   mcc_description
131      7011  Отели, мотели, базы отдыха, сервисы бронирования
    mcc_code                    mcc_description
25      5013  Поставщики грузовиков и запчастей
    mcc_code                                    mcc_description
62      5511  Легковой и грузовой транспорт — продажа, с

In [ ]:
transactions

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN
...,...,...,...,...,...,...
999995,98801178,269 10:29:06,5921,1010,-23016.59,127873
999996,98801178,270 10:26:45,5211,1010,-61380.88,10114209
999997,98801178,270 15:18:53,5921,1010,-11800.27,10105341
999998,98801178,271 12:47:13,5211,1010,-16170.59,10114209


# Задание 2 (3 балла)

Модель - многослойная нейронная сеть минимум с двумя скрытыми слоями и с dropout. Количество нейронов в каждом слое выбираете самостоятельно.
Функции активации, метод оптимизации, скорость обучения, вероятность dropout - выбираете самостоятельно

Модель нужно строить с помощью keras или torch

Настроить параметры модели.

## Вопросы к заданию 2

1. Перечислите все гиперпараметры и их значения вот в таком формате (значения приведены примерные, у вас могут быть другие):

* количество эпох: 5
* скорость обучения: 0.0001
* функция активация на скрытых слоях: ReLU
* и т.д.

2. Сколько у вашей модели параметров?

## Ответы:

# Задание 3 (2 балла)


Проверить качество модели не менее, чем на трёх разных метриках. 

## Вопросы к заданию 3

1. Выведите значения метрик.
2. Как улучшить модель?
3. Для чего может понадобиться предсказание пола клиента по его транзакциям?

## Ответы:

После завершения контрольной работы, дайте ссылку на colab в отдельном followup в своей теме на Piazza. Дайте разрешение на его просмотр.

Не забудьте запустить код в последней ячейке для контроля времени выполнения.

In [ ]:
current_datetime = datetime.now()
print(current_datetime)

2022-12-12 08:22:52.089295
